In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Embedding,Bidirectional,LSTM,TimeDistributed
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import Adam

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ner.csv')

In [ ]:
X=df['Sentence']
Y=df['Tag']


In [ ]:
max_words=36000
tokenizer=Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences=tokenizer.texts_to_sequences(X)

In [ ]:
max_len=110
X_preprocessed=pad_sequences(sequences,maxlen=max_len,padding='post')

In [ ]:
from ast import literal_eval
Y_ready=[]
for sentence_tag in Y:
  Y_ready.append(literal_eval(sentence_tag))

In [ ]:
tags=[]
for y in Y_ready:
  for tag in y:
    if tag not in tags:
      tags.append(tag)

In [ ]:
num_tags=len(tags)

In [ ]:
tags2id={}
for i,tag in enumerate(tags):
  tags2id[tag]=i

In [ ]:
id2tags={}
for key,value in tags2id.items():
  id2tags[value]=key

In [ ]:
def preprocess_y(tags2id,Y_ready):
  Y_preprocessed=[]
  max_len=110
  for y in Y_ready:
    result=[]
    for tag in y:
      result.append(tags2id[tag])
    length=len(result)
    num_O_to_append=max_len-length
    padded_tags=result+([tags2id['O']]*num_O_to_append)
    Y_preprocessed.append(padded_tags)
  return Y_preprocessed


In [ ]:
Y_preprocessed=preprocess_y(tags2id,Y_ready)

In [ ]:
X_train=X_preprocessed[:30000]
X_val=X_preprocessed[30000:37000]
X_test=X_preprocessed[37000:]

In [ ]:
Y_preprocessed=np.array(Y_preprocessed)
Y_train=Y_preprocessed[:30000]
Y_val=Y_preprocessed[30000:37000]
Y_test=Y_preprocessed[37000:]

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((X_train,Y_train))
val_dataset=tf.data.Dataset.from_tensor_slices((X_val,Y_val))
test_dataset=tf.data.Dataset.from_tensor_slices((X_test,Y_test))

In [ ]:
batch_size=128
train_dataset=train_dataset.batch(batch_size)
val_dataset=val_dataset.batch(batch_size)
test_dataset=test_dataset.batch(batch_size)

In [ ]:
model=Sequential()
model.add(Embedding(max_words,300,input_length=110))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(TimeDistributed(Dense(num_tags,activation='softmax')))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              metrics='accuracy',
              optimizer='adam')

In [ ]:
model.fit(train_dataset,validation_data=val_dataset,epochs=10)

Epoch 1/10
235/235 [==============================] - 47s 152ms/step - loss: 0.1892 - accuracy: 0.9656 - val_loss: 0.1138 - val_accuracy: 0.9701
Epoch 2/10
235/235 [==============================] - 9s 39ms/step - loss: 0.0976 - accuracy: 0.9724 - val_loss: 0.0831 - val_accuracy: 0.9751
Epoch 3/10
235/235 [==============================] - 9s 39ms/step - loss: 0.0709 - accuracy: 0.9776 - val_loss: 0.0730 - val_accuracy: 0.9775
Epoch 4/10
235/235 [==============================] - 9s 40ms/step - loss: 0.0578 - accuracy: 0.9808 - val_loss: 0.0695 - val_accuracy: 0.9788
Epoch 5/10
235/235 [==============================] - 9s 40ms/step - loss: 0.0484 - accuracy: 0.9835 - val_loss: 0.0707 - val_accuracy: 0.9790
Epoch 6/10
235/235 [==============================] - 9s 40ms/step - loss: 0.0427 - accuracy: 0.9854 - val_loss: 0.0718 - val_accuracy: 0.9801
Epoch 7/10
235/235 [==============================] - 9s 40ms/step - loss: 0.0383 - accuracy: 0.9868 - val_loss: 0.0744 - val_accuracy: 0.97

In [ ]:
model.evaluate(test_dataset)

86/86 [==============================] - 1s 14ms/step - loss: 0.0776 - accuracy: 0.9799


[0.07755501568317413, 0.979855477809906]

In [ ]:
words2id=tokenizer.word_index
id2words={}
for key,value in words2id.items():
  id2words[value]=key

In [ ]:
def prediction(model,test_sentence,id2tags,id2words):
  test_sentence=test_sentence.reshape(1,110)
  sentence=test_sentence[test_sentence>0]
  word_list=[]
  for id in list(sentence):
    word_list.append(id2words[id])
  original_sentence=' '.join(word_list)
  len_original_sentence=len(original_sentence)

  prediction=model.predict(test_sentence)
  prediction=np.argmax(prediction[0],axis=1)
  prediction=prediction[:len_original_sentence]

  predicted_tags=[]
  for tag_id in list(prediction):
    predicted_tags.append(id2tags[tag_id])
  return original_sentence,predicted_tags

In [ ]:
original_sentence,predicted_tags=prediction(model,X_test[0],id2tags,id2words)

1/1 [==============================] - 0s 24ms/step


In [ ]:
print(original_sentence)

kosovo 's delegation for possible talks on the future of the united nations run province has met for the first time and stressed it would seek independence for the largely ethnic albanian region


In [ ]:
print(predicted_tags)

['B-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'I-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
